<a href="https://colab.research.google.com/github/meteve/NLP_project/blob/master/scripts/who_wrote_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Who wrote this : a framework for French novelist identification

In [1]:
import os
import re
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count

import unidecode
import urllib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, confusion_matrix

## Data preprocessing

In [2]:
# Import train data
train_df = pd.read_csv('../data/corpus_train_features_NER.csv', index_col=0)
X_train_ner = train_df['paragraph_ner'].values
X_train = train_df['paragraph'].values
y_labels_train = train_df['author'].values

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_labels_train)

In [3]:
# Import test data
test_df = pd.read_csv('../data/corpus_test_NER.csv', index_col=0)
X_test_ner = test_df['paragraph_ner'].values
X_test = test_df['paragraph'].values
y_labels_test = test_df['author'].values
y_test = le.transform(y_labels_test)

In [ ]:
def import_stopwords():
    """Download and import French stopwords."""
    URL = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-fr/master/stopwords-fr.txt'
    response = urllib.request.urlopen(URL)
    stopwords = response.read().decode('utf-8').splitlines()
    stopwords = [unidecode.unidecode(x) for x in stopwords]
    stopwords.append('quelqu') # Make stopwords consistent with tokenization
    return stopwords

# Import french stopwords
#with open('models/stopwords-fr.txt') as f:
 #   stopwords = f.read().splitlines()
stopwords = import_stopwords()
stopwords = [unidecode.unidecode(x) for x in stopwords]

## Baseline TF-IDF model with and without NER

In [4]:
# ML pipeline : TF-IDF + SVM classifier

tfidf_vecto = TfidfVectorizer()
clf = LinearSVC()

tfidf_pipeline = Pipeline([
                           ('tf-idf', tfidf_vecto),
                           ('SVC', clf)
])

Compare train and test scores with and without NER procedure.

In [6]:
# Compute predictions on train and train score
y_train_pred = tfidf_pipeline.predict(X_train)
tfidf_test_score = f1_score(y_train, y_train_pred, average='micro')

y_train_pred_ner = tfidf_pipeline_ner.predict(X_train_ner)
tfidf_test_score_ner = f1_score(y_train, y_train_pred_ner, average='micro')

print('F1 score on train set with TF-IDF :', 
      tfidf_test_score.round(3))
print('F1 score with NER procedure on train set with TF-IDF :', 
      tfidf_test_score_ner.round(3))

F1 score on train set with TF-IDF : 0.972
F1 score with NER procedure on train set with TF-IDF : 0.973


In [7]:
# Compute predictions and test score
y_pred = tfidf_pipeline.predict(X_test)
tfidf_test_score = f1_score(y_test, y_pred, average='micro')

y_pred_ner = tfidf_pipeline_ner.predict(X_test_ner)
tfidf_test_score_ner = f1_score(y_test, y_pred_ner, average='micro')

print('F1 score on test set with TF-IDF :', 
      tfidf_test_score.round(3))
print('F1 score with NER procedure on test set with TF-IDF :', 
      tfidf_test_score_ner.round(3))

F1 score on test set with TF-IDF : 0.501
F1 score with NER procedure on test set with TF-IDF : 0.487


Confusion matrix

In [8]:
def get_conf_matrix_df(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred)
    conf_matrix = pd.DataFrame(conf_matrix, columns=list(le.classes_), index=list(le.classes_))
    
    row_percentages = []
    for i in range(0,10):
        perc = round(conf_matrix.iloc[i,i]/sum(conf_matrix.iloc[0:10,i]), 3)
        row_percentages.append(perc)
    
    row_percentages = pd.DataFrame([row_percentages], columns=list(le.classes_))
    
    col_percentages = []
    for i in range(0,10):
        perc = round(conf_matrix.iloc[i,i]/sum(conf_matrix.iloc[i,0:10]), 3)
        col_percentages.append(perc)
        
    
    conf_matrix['recall'] = col_percentages
    conf_matrix = conf_matrix.append(row_percentages)
    
    return(conf_matrix)

In [9]:
conf_matrix_train = get_conf_matrix_df(y_train, y_train_pred)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [10]:
conf_matrix_test = get_conf_matrix_df(y_test, y_pred)

In [11]:
conf_matrix_test

,Balzac,Daudet,Dumas,Flaubert,Hugo,Maupassant,Stendhal,Verne,Vigny,Zola,col_percentages
Balzac,730.000,29.000,409.000,116.000,129.000,101.000,546.000,39.000,33.000,53.000,0.334
Daudet,21.000,79.000,170.000,91.000,88.000,72.000,55.000,70.000,17.000,52.000,0.110
Dumas,99.000,43.000,3458.000,108.000,313.000,152.000,230.000,84.000,38.000,48.000,0.756
Flaubert,26.000,8.000,155.000,624.000,125.000,87.000,96.000,21.000,11.000,58.000,0.515
Hugo,40.000,47.000,688.000,120.000,1521.000,107.000,309.000,146.000,42.000,32.000,0.498
Maupassant,18.000,18.000,126.000,50.000,75.000,211.000,36.000,33.000,9.000,42.000,0.341
Stendhal,27.000,9.000,208.000,33.000,39.000,29.000,897.000,22.000,21.000,13.000,0.691
Verne,12.000,2.000,181.000,39.000,56.000,24.000,52.000,421.000,16.000,12.000,0.517
Vigny,18.000,75.000,159.000,17.000,73.000,28.000,63.000,35.000,85.000,14.000,0.150
Zola,54.000,56.000,346.000,118.000,386.000,173.000,147.000,166.000,57.000,508.000,0.253


## FastText pre-trained embeddings + averaging

In [0]:
# Use sklearn preprocessing pipeline to ensure results comparability
preprocessor = tfidf_vecto.build_analyzer()

In [0]:
# Import Fasttext French word vectors
fasttext = FastText.load_fasttext_format('models/fasttext.fr.300.bin')

In [0]:
def text_to_wv_fasttext(text):
    """Compute average of FastText's word vectors for a given text."""
    if text:
        tokens = preprocessor(text)
        wv_mat = np.zeros((len(tokens), fasttext.vector_size))
        for i, tok in enumerate(tokens):
            try:
                wv_mat[i] = fasttext.wv[tok]
            except KeyError:
                pass
        text_vec = wv_mat.mean(axis=0)
    else:
        text_vec = np.zeros(fasttext.vector_size)
    return text_vec

In [0]:
def preprocess_corpus_fasttext(corpus):
    """Parallelize preprocessing and document vectors computation."""
    with Pool(N_CORES) as p:
        corpus_prepro = p.map(text_to_wv_fasttext, list(corpus))
    return np.array(corpus_prepro)

In [0]:
class TextToWV(BaseEstimator, TransformerMixin):
    """Enable to use preprocessing function in a sklearn pipeline."""
    def __init__(self, preprocessor):
        self.preprocessor = preprocessor

    def fit(self, X, y=None):
        return(self)

    def transform(self, X):
        return self.preprocessor(X)

In [0]:
fasttext_pipeline = Pipeline([
                              ('fasttext_average', TextToWV(preprocess_corpus_fasttext)),
                              ('SVC', clf)
])

In [0]:
# Preprocessing + training
fasttext_pipeline = fasttext_pipeline.fit(X_train, y_train)

In [0]:
# Compute predictions and test score
y_pred = fasttext_pipeline.predict(X_test)
fasttext_test_score = f1_score(y_test, y_pred, average='micro')
print('F1 score on test set with pre-trained FastText + averaging :',
      fasttext_test_score.round(2))

F1 score on test set with pre-trained FastText + averaging : 0.37


## CamemBERT

In [0]:
# Import Camembert model
camembert = CamembertModel.from_pretrained('models/camembert.v0')
camembert.eval()
CAMEMBERT_WV_SIZE = camembert.extract_features(camembert.encode('test string')).shape[2]

loading archive file models/camembert.v0
| dictionary: 32004 types


In [0]:
test = [camembert.encode(x) for x in X_train[:100]]
max([len(x) for x in test])

2334

In [0]:
def text_to_wv_camembert(text):
    """Compute average of CamemBERT's word vectors for a given text."""
    if text:
        sentences = text.split('.')
        sentences = [s for s in sentences if s]
        tokens_ind = camembert.encode(*sentences)
        text_features = camembert.extract_features(tokens_ind)
        text_vec = text_features.squeeze(0).detach().numpy().mean(axis=0)
    else:
        text_vec = np.zeros(CAMEMBERT_WV_SIZE)
    return text_vec

In [0]:
def preprocess_corpus_camembert(corpus):
    """"""
    corpus_prepro = []
    for i, text in enumerate(corpus):
        corpus_prepro.append(text_to_wv_camembert(text))
        print(i)
    return np.array(corpus_prepro)

In [0]:
camembert_pipeline = Pipeline([
                              ('camembert_average', TextToWV(preprocess_corpus_camembert)),
                              ('SVC', clf)
])

In [0]:
# Preprocessing + training
camembert_pipeline = camembert_pipeline.fit(X_train, y_train)

0
1
2


ValueError: ignored